# ENGRI 1120 Discussion 1: Fun With Total and Species Mass Balances

## Introduction
Suppose we interested in tracking the mass of each chemical component in a mixture of $\mathcal{M}$ components for a chemical reaction process e.g., the starting material, the final product and perhaps some by-products flowing into and from a chemical reactor unit. Then, we can write the _species mass balance_ for component $i$ as:

$$\sum_{s\in\mathcal{S}}\nu_{s}\dot{m}_{s,i} + \dot{m}_{G,i} = \frac{dm_{i}}{dt}\qquad{i=1,2,\dots,\mathcal{M}}$$

where $\mathcal{S}$ denotes the set of streams associated with the chemical reactor unit, $\dot{m}_{s,i}$ denotes the mass flow rate of component $i$ in stream $s$ (units: kg per hour), $\dot{m}_{G,i}$ denotes the generation rate for species $i$ (units: kg per hour) and $\nu_{s}$ denotes the stream direction paramater.

## Materials and Methods

#### Project setup

In [1]:
import Pkg; Pkg.activate("."); Pkg.resolve(); Pkg.instantiate();

  Activating project at `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/labs/lab-1-fun-w-mass-balances`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/labs/lab-1-fun-w-mass-balances/Project.toml`
  No Changes to `~/Desktop/julia_work/ENGRI-1120-IntroToChemE-Example-Notebooks/labs/lab-1-fun-w-mass-balances/Manifest.toml`


In [2]:
using Distributions
using TOML

#### Load Julia code library

In [3]:
include("ENGRI-1120-Lab-1-CodeLib.jl"); # this call loads functions into the notebook so that we can use them

#### Example: solve the dynamic system of species mass balances
The simulation details are stored in a [TOML](https://toml.io/en/) file in the `configuration` directory. The configuration file holds information about the rate and composition of the feed coming into the mixing unit and some other parameters that are required for the mixing simulation. 

To load the configuration file:
* Specify the path to the configuration file using the `joinpath` command
* Pass this path to the `build_config` command (loaded from `ENGRI-1120-Lab-1-CodeLib.jl`) which parses the [TOML](https://toml.io/en/) file, and returns a [Julia Dictionary](https://docs.julialang.org/en/v1/base/collections/#Base.Dict) holding information about the simulation.

In [5]:
# Setup the path to the file we want to load -
path_to_config_file = joinpath(pwd(), "configuration", "Example-Tank.toml")

# load the config file with simulation details -
config_dict = build_config(path_to_config_file);

The species mass balance equations are solved using the `simulate_species_mass_balances` function (loaded from `ENGRI-1120-Lab-1-CodeLib.jl`). This function takes the configuration dictionary and returns the time array `T` and the state array `X`.  

In [6]:
(T,X) = simulate_species_mass_balances(config_dict; σ = 0.0); # solve the species balances, with no process noise

## Problems

### A) How long does it take the system to reach steady state?

### B) At steady state, is conservation of mass true?

### C) The client wants a different mixing ratio, how can we do that?